# Automating Coconut Maturity Detection with Drone Technology

# Import Libraries

In [ ]:
import os
import glob as glob
import matplotlib.pyplot as plt
import cv2
import requests
import random
import numpy as np

np.random.seed(42)

#Dataset preparation

The Coconut-dataset from Roboflow for training the custom YOLOv5 object detector.
The code to download the dataset you can get from https://universe.roboflow.com/nit-calicut/coconut-veirf/dataset/5#. Do not share this snippet beyond your team, it contains a private
key that is tied to your Roboflow account. Replace the XXXX with the appropriate link

In [59]:
if not os.path.exists('roboflow.zip'):
    !curl -L "https://universe.roboflow.com/ds/XXXX" > roboflow.zip;
    !unzip roboflow.zip;


Create a folder inference if it doesnt exist. In this folder we copy all image files collected from drone of coconut farm

In [60]:
if not os.path.exists('inference'):
    !mkdir inference
    from google.colab import drive
    drive.mount('/content/drive')
    !cp /content/drive/My\ Drive/inference/*.jpg /content/inference/

The dataset is organized into three folders: train, valid (validation), and test. Each of these folders contains two subfolders: images and labels. Metadata about the dataset is stored in data.yaml.

The dataset is structured in the following manner:

1.   data.yaml
2.   test
*   images
*   labels
3. train
*   images
*   labels
4. valid
*   images
*   labels

The labels directory contains label.txt files that consists of annotation data. YOLOv5 Pytorch normalised format is as follows:

"class_label center_x center_y width, height"

## Convert bounding boxes in YOLO format to xmin, ymin, xmax, ymax

**Normalization** is the process of scaling data to a specific range, typically between 0 and 1. This is done to ensure that all features contribute equally to the model's learning process, preventing features with larger values from dominating those with smaller values. It also helps improve the stability and speed of training.

**In the context of YOLOv5's bounding box annotations:**

**center_x and center_y:** The center coordinates of the bounding box are normalized by dividing the pixel coordinates by the image width and height, respectively.
**width and height:** The width and height of the bounding box are normalized by dividing the pixel dimensions by the image width and height, respectively.

This ensures all values are within the 0-1 range, regardless of the original image size. Let's say you have an image that is 640 pixels wide and 480 pixels high. You want to annotate a dog in the image. The bounding box around the dog has the following pixel coordinates:

**Top-left corner:** (100, 150) Bottom-right corner: (400, 350) To calculate the normalized YOLOv5 format:

**Center_x:** (100 + 400) / 2 = 250. 250 / 640 (image width) = 0.39 Center_y: (150 + 350) / 2 = 250. 250 / 480 (image height) = 0.52 Width: 400 - 100 = 300. 300 / 640 (image width) = 0.47 Height: 350 - 150 = 200. 200 / 480 (image height) = 0.42 Assuming the class_label for "dog" is 0, the annotation line in the label.txt file would be:

0 0.39 0.52 0.47 0.42

Visualize a Few Ground Truth Images Before moving forward, let's check out few of the ground truth images. The current annotations in the text files are in normalized [x_center, y_center, width, height] format. Let's write a function that will convert it back to [x_min, y_min, x_max, y_max] format.

Before proceeding, let's examine some of the correct, labeled images (ground truth). The image labels are currently in a normalized format representing the center point, width, and height of a bounding box. We need to convert these labels to a format representing the top-left and bottom-right corners of the bounding box.

The dataset YAML file, named data.yaml, includes important information for training a model. It specifies the locations of the training, validation, and test images, as well as their corresponding labels.

**Key components of the file are:**

* *train:* Path to the directory containing training images (../train/images).
**test:* Path to the directory for test images (../test/images).
**val:* Path to the directory for validation images (../valid/images).
**nc:* Number of classes in the dataset (1 class).
**names:* List of class names, which in this case contains only one class: 'drone'.
This structured format helps in organizing data for machine learning tasks.

In [61]:
# Function to convert bounding boxes in YOLO format to xmin, ymin, xmax, ymax.
def yolo2bbox(bboxes):
    xmin, ymin = bboxes[0]-bboxes[2]/2, bboxes[1]-bboxes[3]/2
    xmax, ymax = bboxes[0]+bboxes[2]/2, bboxes[1]+bboxes[3]/2
    return xmin, ymin, xmax, ymax

In [56]:
class_names = ['dry', 'green', 'tender']

#Visualization

##Render bounding box around the objects according to the annotated data.

In [57]:
def plot_box(image, bboxes, labels):
    # Need the image height and width to denormalize the bounding box coordinates.
    h, w = image.shape[:2]

    for box_num, box in enumerate(bboxes):
        # Obtain top_left and bottom_right corners.
        x1, y1, x2, y2 = yolo2bbox(box)
        # denormalize the coordinates.
        xmin = int(x1*w)
        ymin = int(y1*h)
        xmax = int(x2*w)
        ymax = int(y2*h)
        # Width and height of the bounding box.
        width = xmax - xmin
        height = ymax - ymin

        class_name = class_names[int(labels[box_num])]

        # Render bounding box around the object.
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), color=(0,0,255), thickness=max(2, int(w/275)))

        # Render labels.
        font_scale = min(1, max(4,int(w/500)))
        font_thickness = min(1, max(10,int(w/50)))
        p1, p2 = (int(xmin), int(ymin)), (int(xmax), int(ymax))

        # Text width and height.
        tw, th = cv2.getTextSize(class_name, 0, fontScale=font_scale, thickness=font_thickness)[0]

        p2 = p1[0] + tw, p1[1] + -th - 10

        cv2.rectangle(image, p1, p2, color=(255,0,0), thickness=-1)
        cv2.putText(image, class_name, (xmin+1, ymin-10), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), font_thickness, cv2.LINE_AA)

    return image

##Visualize random images with the bounding boxes

In [ ]:
def visualize(image_paths, label_paths, num_samples):
    all_training_images = glob.glob(image_paths)
    all_training_labels = glob.glob(label_paths)
    all_training_images.sort()
    all_training_labels.sort()

    num_images = len(all_training_images)

    plt.figure(figsize=(20, 17))
    for i in range(num_samples):
        j = random.randint(0, num_images-1)
        image = cv2.imread(all_training_images[j])
        with open(all_training_labels[j], 'r') as f:
            bboxes = []
            labels = []
            label_lines = f.readlines()
            for label_line in label_lines:
                label = label_line[0]
                bbox_string = label_line[2:]
                bbox_string
                x_c, y_c, w, h = bbox_string.split(' ' )
                x_c = float(x_c)
                y_c = float(y_c)
                w = float(w)
                h = float(h)
                bboxes.append([x_c, y_c, w, h])
                labels.append(label)
        result_image = plot_box(image, bboxes, labels)
        plt.subplot(2, 2, i+1)
        plt.imshow(result_image[:, :, ::-1])
        plt.axis('off')
    plt.subplots_adjust(wspace=0)
    plt.tight_layout()
    plt.show()
# Visualize a few training images.
visualize(
    image_paths='train/images/*',
    label_paths='train/labels/*',
    num_samples=4,
)

#Training

##Clone YOLOV5 Repository

In [67]:
if not os.path.exists('yolov5'):
    !git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 17022, done.
remote: Total 17022 (delta 0), reused 0 (delta 0), pack-reused 17022 (from 1)
Receiving objects: 100% (17022/17022), 15.62 MiB | 32.58 MiB/s, done.
Resolving deltas: 100% (11694/11694), done.


In [68]:
%cd yolov5/
!pwd

/content/yolov5/yolov5/yolov5
/content/yolov5/yolov5/yolov5


In [69]:
!pip install -r requirements.txt

## Training model

Before starting your training, sign up for a Weights &amp; Biases (wandb) account at wandb.ai using your
GitHub account. If you already have a wandb account, select the &quot;use existing account&quot; option,
authenticate, and obtain your API key. You&#39;ll be prompted to enter this API key during the training
setup process.

In [66]:
!python train.py --data ../data.yaml --weights yolov5m.pt --img 640 --epochs {50} --batch-size 32
#!python train.py --data ../data.yaml --weights yolov5m.pt --img 640 --epochs 25 --batch-size 32 --device 0 --nosave!

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/__init__.py", line 25, in <module>
    from .artifacts.artifact import Artifact
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/artifacts/artifact.py", line 37, in <module>
    from wandb.apis.normalize import normalize_exceptions
  File "/usr/local/lib/python3.10/dist-packages/wandb/apis/__init__.py", line 44, in <module>
    from .public import Api as PublicApi

#Testing & Inference

In [ ]:
!python val.py --task 'test' --weights runs/train/exp/weights/best.pt --data ../data.yaml

In this section, we will carry out inference on unseen images and videos from the internet. Note that the inferece_results folder will change with every run. Make sure to change the path if you are running the block more than once.

In [ ]:
# Run inference.
!python detect.py --source ../inference --weights runs/train/exp/weights/best.pt --project inference_results --name .
plt.figure(figsize=(25,15))
for i in range(3):
  img = cv2.imread('./inference_results/coconut-test{}.jpg'.format(i+1))
  plt.subplot(1, 3, i+1);
  plt.imshow(img[...,::-1]);

##Download Model

In [ ]:
from google.colab import files
files.download('runs/train/exp2/weights/best.pt')